In [3]:
from skilift import GTFSFeed, get_stop_vertex, read_osm, \
StreetData, OnEarthSurfaceVertex, TransitEdgeProvider, StreetEdgeProvider, \
NodeRef, StreetNodeVertex, TransitStreetConnectionEdgeProvider

In [4]:
import pandas as pd
from zipfile import ZipFile
from collections import defaultdict
from typing import Dict, Set, Tuple, Iterable, Iterator, Any
import numpy as np
from shapely.geometry import Point, LineString, MultiLineString
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [5]:
fn = "/home/bmander/skilift_data/transit/redding.zip"

feed = GTFSFeed(fn)
transit_edge_provider = TransitEdgeProvider(feed)

In [6]:
# get epoch time on Aptil 19, 2023 at 1:00pm pacific time
t0 = pd.Timestamp("2023-04-19 13:00:00", tz="America/Los_Angeles")
t0

Timestamp('2023-04-19 13:00:00-0700', tz='America/Los_Angeles')

In [7]:
node0 = get_stop_vertex(feed, "Downtown Transit Center", t0)
node0

AtStopvertex(stop_id:2000, datetime:2023-04-19 13:00:00-07:00)

In [8]:
node0.stop_id

2000

In [9]:
node1 = transit_edge_provider.outgoing(node0)[0].vertex
node1

DepartureVertex(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [10]:
node2 = transit_edge_provider.outgoing(node0)[0].vertex
node2

DepartureVertex(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [11]:
transit_edge_provider.outgoing(node2)

[Edge(vertex=ArrivalVertex(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00), weight=600.0)]

In [12]:
osm_filename = "/home/bmander/skilift_data/street/redding.pbf"


In [13]:
elev_geotiff_filename = "/home/bmander/skilift_data/elevation/USGS_13_n41w123_20210624.tif"

In [14]:
elev_dataset = StreetData(osm_filename, elev_geotiff_filename)

Reading OSM file...done
Indexing ways...done
Getting node elevations...done
Creating spatial index...done


In [15]:
nd_points = elev_dataset.nodes.values()
lon = sum([x[0] for x in nd_points])/len(nd_points)
lat = sum([x[1] for x in nd_points])/len(nd_points)
lon, lat

(-122.35873694849425, 40.584176490162584)

In [16]:
node0 = OnEarthSurfaceVertex(lon, lat, t0)
node0

OnEarthSurfaceNode(-122.35873694849425, 40.584176490162584, 2023-04-19 13:00:00-07:00)

In [17]:
street_edge_provider = StreetEdgeProvider(elev_dataset)
v0 = street_edge_provider.outgoing(node0)[0].vertex
v0

MidStreetNode(SegmentRef(way_id=774909443, segment_index=1), 0.573216480698101, 2023-04-19 13:00:50.220285374-07:00)

In [18]:
way = street_edge_provider.osm_data.ways[774909443]

In [19]:
street_edge_provider.osm_data.node_refs[way.nds[5]]

{NodeRef(way_id=675408601, node_index=0),
 NodeRef(way_id=774909443, node_index=5)}

In [20]:
v0.segment_ref

SegmentRef(way_id=774909443, segment_index=1)

In [21]:
street_edge_provider.osm_data.adj_vertex_node(NodeRef(v0.segment_ref.way_id, 1), search_forward=False)

0

In [22]:
seg_start = 1
seg_end = 0

In [23]:
way.nds

[5797734026, 5797733850, 5797733851, 5797733852, 5797733953, 5797734025]

In [24]:
way.nds[seg_start: None: -1]

[5797733850, 5797734026]

In [25]:
edges = street_edge_provider.outgoing(v0)
edges

[Edge(vertex=StreetNodeVertex(5797734025, 2023-04-19 13:01:59.235587877-07:00), weight=69.0153025031656),
 Edge(vertex=StreetNodeVertex(5797734026, 2023-04-19 13:01:46.456623367-07:00), weight=56.236337993785625)]

In [26]:
edges = street_edge_provider.outgoing( edges[0].vertex )
edges

[Edge(vertex=StreetNodeVertex(5797734026, 2023-04-19 13:04:04.487228373-07:00), weight=125.25164049695098),
 Edge(vertex=StreetNodeVertex(5797733955, 2023-04-19 13:04:57.860111873-07:00), weight=53.37288350041744)]

In [27]:
edges = street_edge_provider.outgoing( edges[1].vertex )
edges

[Edge(vertex=StreetNodeVertex(5797734025, 2023-04-19 13:05:51.232995373-07:00), weight=53.37288350041742),
 Edge(vertex=StreetNodeVertex(5797733965, 2023-04-19 13:07:21.984013803-07:00), weight=90.75101843063389)]

In [28]:
street_edge_provider.outgoing( StreetNodeVertex(5797734025, pd.Timestamp(0)) )

[Edge(vertex=StreetNodeVertex(5797734026, 1970-01-01 00:02:05.251640496), weight=125.25164049695098),
 Edge(vertex=StreetNodeVertex(5797733955, 1970-01-01 00:02:58.624523996), weight=53.37288350041744)]

In [29]:
street_edge_provider.outgoing( StreetNodeVertex(5797733955, pd.Timestamp(0)) )

[Edge(vertex=StreetNodeVertex(5797734025, 1970-01-01 00:00:53.372883500), weight=53.37288350041742),
 Edge(vertex=StreetNodeVertex(5797733965, 1970-01-01 00:02:24.123901930), weight=90.75101843063389)]

In [30]:
street_edge_provider.outgoing( StreetNodeVertex(5797734026, pd.Timestamp(0)) )


[Edge(vertex=StreetNodeVertex(5797734025, 1970-01-01 00:02:05.251640496), weight=125.25164049695098),
 Edge(vertex=StreetNodeVertex(5797733849, 1970-01-01 00:02:12.271282712), weight=7.019642216331847)]

In [31]:
tscep = TransitStreetConnectionEdgeProvider(feed, elev_dataset)

In [34]:
tscep.node_to_stops

defaultdict(set,
            {4676419906: {(1000,
               SegmentLinearRef(segment=SegmentRef(way_id=473558836, segment_index=28), offset=0.29250983596728247))},
             4676419907: {(1000,
               SegmentLinearRef(segment=SegmentRef(way_id=473558836, segment_index=28), offset=0.29250983596728247))},
             4665396134: {(1001,
               SegmentLinearRef(segment=SegmentRef(way_id=472328220, segment_index=0), offset=0.33915121469449744))},
             4665396133: {(1001,
               SegmentLinearRef(segment=SegmentRef(way_id=472328220, segment_index=0), offset=0.33915121469449744))},
             4676349744: {(1002,
               SegmentLinearRef(segment=SegmentRef(way_id=677196775, segment_index=66), offset=0.24948472809560712))},
             4676349745: {(1002,
               SegmentLinearRef(segment=SegmentRef(way_id=677196775, segment_index=66), offset=0.24948472809560712))},
             5409838583: {(1003,
               SegmentLinearRef(segment=